In [1]:
from utils import Utils 
import pandas as pd
utils=Utils()

# BASE PLAYER_SCRAPER

In [2]:
df = utils.cargar_excel("Mundial_Historic_players.xlsx")

In [3]:
df

,pais,Nombre_jugador,Posicion,Edad,Partidos_jugados,Club,Año,Campeón,pais_AND_Campeón,Continente
0,Inglaterra,Gordon Banks,Portero,28,27,Leicester City,1966,Inglaterra,1,Europa
1,Inglaterra,George Cohen,Defensa,26,24,Fulham,1966,Inglaterra,1,Europa
2,Inglaterra,Ray Wilson,Defensa,31,45,Everton,1966,Inglaterra,1,Europa
3,Inglaterra,Nobby Stiles,Centrocampista,24,14,Manchester United,1966,Inglaterra,1,Europa
4,Inglaterra,Jack Charlton,Defensa,31,16,Leeds United,1966,Inglaterra,1,Europa
5,Inglaterra,Bobby Moore,Defensa,25,41,West Ham United,1966,Inglaterra,1,Europa
6,Inglaterra,Alan Ball,Centrocampista,21,10,Blackpool,1966,Inglaterra,1,Europa
7,Inglaterra,Jimmy Greaves,Delantero,26,51,Tottenham Hotspur,1966,Inglaterra,1,Europa
8,Inglaterra,Bobby Charlton,Centrocampista,28,68,Manchester United,1966,Inglaterra,1,Europa


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from bs4 import BeautifulSoup

def buscar_wikipedia_jugadores(df):
    """
    Busca en DuckDuckGo los nombres de los jugadores y obtiene su enlace de Wikipedia si existe.

    Parámetros:
    df (pd.DataFrame): DataFrame con la columna 'Nombre_jugador'.

    Retorna:
    pd.DataFrame: DataFrame con 'Nombre_jugador' y 'wiki_link'.
    """
    # Configuración del WebDriver
    driver = webdriver.Chrome()
    wait = WebDriverWait(driver, 10)

    resultados = []  # Lista para almacenar los resultados

    for nombre in df["Nombre_jugador"]:
        try:
            # Buscar en DuckDuckGo
            query = f"{nombre} site:wikipedia.org"
            url_busqueda = f"https://www.duckduckgo.com/?q={query.replace(' ', '+')}"
            driver.get(url_busqueda)
            
            # Esperar a que se cargue la página
            time.sleep(2)
            soup = BeautifulSoup(driver.page_source, "html.parser")

            # Buscar el primer enlace de Wikipedia en los resultados
            link_wiki = None
            for enlace in soup.find_all("a", href=True):
                href = enlace["href"]
                if "wikipedia.org/wiki/" in href:
                    link_wiki = href
                    break

            # Agregar el resultado
            resultados.append({"Nombre_jugador": nombre, "wiki_link": link_wiki if link_wiki else "No encontrado"})

        except Exception as e:
            print(f"Error con {nombre}: {e}")
            resultados.append({"Nombre_jugador": nombre, "wiki_link": "Error"})

    # Cerrar el navegador
    driver.quit()

    # Crear DataFrame con los resultados
    df_links_names = pd.DataFrame(resultados)

    return df_links_names


In [5]:
df_links_names=buscar_wikipedia_jugadores(df)

In [6]:
df_links_names

,Nombre_jugador,wiki_link
0,Gordon Banks,/?q=Gordon+Banks+site%3Awikipedia.org&ia=web
1,George Cohen,/?q=George+Cohen+site%3Awikipedia.org&ia=web
2,Ray Wilson,/?q=Ray+Wilson+site%3Awikipedia.org&ia=web
3,Nobby Stiles,/?q=Nobby+Stiles+site%3Awikipedia.org&ia=web
4,Jack Charlton,/?q=Jack+Charlton+site%3Awikipedia.org&ia=web
5,Bobby Moore,/?q=Bobby+Moore+site%3Awikipedia.org&ia=web
6,Alan Ball,/?q=Alan+Ball+site%3Awikipedia.org&ia=web
7,Jimmy Greaves,/?q=Jimmy+Greaves+site%3Awikipedia.org&ia=web
8,Bobby Charlton,/?q=Bobby+Charlton+site%3Awikipedia.org&ia=web
